# Trợ lý AI tự động cập nhật tin tức và đánh giá

In [31]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other
import get_artical
import analyze_article
import gemini_model

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(get_artical)
importlib.reload(analyze_article)
importlib.reload(gemini_model)

from import_default import *
from import_database import *
from import_other import *
from get_artical import *
from analyze_article import *
from gemini_model import *

In [32]:
get_gemini_models()

Đang lấy danh sách model...

=== GEMINI 2.0 FLASH MODELS (NO LITE) ===
gemini-2.0-flash
gemini-2.0-flash-001
gemini-2.0-flash-exp
gemini-2.0-flash-exp-image-generation
gemini-2.0-flash-preview-image-generation

=== GEMINI 2.5 FLASH MODELS (NO LITE) ===
gemini-2.5-flash
gemini-2.5-flash-preview-04-17
gemini-2.5-flash-preview-05-20
gemini-2.5-flash-preview-tts

=== GEMINI 2.0 THINKING MODELS ===
gemini-2.0-flash-thinking-exp
gemini-2.0-flash-thinking-exp-01-21
gemini-2.0-flash-thinking-exp-1219

=== GEMINI 2.5 THINKING MODELS ===
gemini-2.5-flash-preview-04-17-thinking


In [ ]:
genai.configure(api_key=load_env("GEMINI_API"))

gemini_20_flash = genai.GenerativeModel('gemini-2.0-flash')
gemini_25_flash = genai.GenerativeModel('gemini-2.5-flash')
gemini_25_thinking = genai.GenerativeModel('gemini-2.5-flash-preview-04-17-thinking')

model_dict = {
    'gemini-2.0-flash': gemini_20_flash,
	'gemini-2.5-flash': gemini_25_flash,
	'gemini-2.5-thinking': gemini_25_thinking
}

In [3]:
vietstock_rss_url_dict = {
    'trong_nuoc': {
        'vi_mo': 'https://vietstock.vn/761/kinh-te/vi-mo.rss',
    },
    'quoc_te': {
        'tai_chinh_quoc_te': 'https://vietstock.vn/772/the-gioi/tai-chinh-quoc-te.rss',
        'chung_khoan_the_gioi': 'https://vietstock.vn/773/the-gioi/chung-khoan-the-gioi.rss',
    },
    'doanh_nghiep': {
        'hoat_dong_kinh_doanh': 'https://vietstock.vn/737/doanh-nghiep/hoat-dong-kinh-doanh.rss',
    }
}

cafef_rss_url_dict = {
    'trong_nuoc': {
        'kinh_te_vi_mo': 'https://cafef.vn/vi-mo-dau-tu.chn',
        'ngan_hang': 'https://cafef.vn/tai-chinh-ngan-hang.chn',
        'chung_khoan': 'https://cafef.vn/thi-truong-chung-khoan.chn',
        'bat_dong_san': 'https://cafef.vn/bat-dong-san.chn',
    },
    'quoc_te': {
        'tai_chinh_quoc_te': 'https://cafef.vn/tai-chinh-quoc-te.chn',
    },
    'doanh_nghiep': {
        'doanh_nghiep': 'https://cafef.vn/doanh-nghiep.chn',
    }
}

In [4]:
vietstock_count_dict = {
    'trong_nuoc': {
        'vi_mo': 2,
    },
    'quoc_te': {
        'tai_chinh_quoc_te': 3,
        'chung_khoan_the_gioi': 3,
    },
    'doanh_nghiep': {
        'hoat_dong_kinh_doanh': 5,
    }
}

cafef_count_dict = {
    'trong_nuoc': {
        'kinh_te_vi_mo': 2,
        'ngan_hang': 2,
        'chung_khoan': 2,
        'bat_dong_san': 2,
    },
    'quoc_te': {
        'tai_chinh_quoc_te': 4,
    },
    'doanh_nghiep': {
        'doanh_nghiep': 5,
    }
}

In [5]:
full_news_df_dict = {}

# Khởi tạo dictionary với list rỗng cho mỗi news_type
for news_type in ['trong_nuoc', 'quoc_te', 'doanh_nghiep']:
    full_news_df_dict[news_type] = []

# Lấy tin từ VietStock
for news_type, url_dict in vietstock_rss_url_dict.items():
    for category_name, url in url_dict.items():
        max_entries = vietstock_count_dict[news_type][category_name]
        feed = feedparser.parse(url)
        
        # Lấy các bài mới nhất theo max_entries
        for entry in feed.entries[:max_entries]:
            # Get both content and image URL
            content, image_url = get_article_vietstock(entry['id'])
            
            # Lấy thời gian đăng bài từ RSS feed
            published_time = ""
            if hasattr(entry, 'published') and entry.published:
                published_time = entry.published
                
			# Tóm tắt content
            model_name_list = list(model_dict.keys())
            for attempt in range(6):
                model_name = model_name_list[attempt % len(model_name_list)]
                try:
                    summary_content = summary_article(model_dict[model_name], content, news_type)
                    break
                except Exception as e:
                    if attempt == 5: raise Exception(f"Lỗi khi tóm tắt bài viết: {e}")
            
            full_news_df_dict[news_type].append({
                'source': 'VietStock',
                'title': entry['title'], 
                'content': summary_content,
                'image_url': image_url,
                'article_url': entry['id'],
                'published_time': published_time,
            })
            time.sleep(1)
        print(f"Thành công lấy tin từ VietStock: {category_name} ({url})")

# Lấy tin từ CafeF (cấu trúc giống VietStock)
for news_type, url_dict in cafef_rss_url_dict.items():
    for category_name, url in url_dict.items():
        max_entries = cafef_count_dict[news_type][category_name]
        # Lấy danh sách bài viết từ trang danh mục (tạo feed giả)
        feed_entries = get_cafef_articles_list(url, max_entries)
        
        # Lấy các bài mới nhất theo max_entries
        for entry in feed_entries[:max_entries]:
            # Get both content and image URL (giống VietStock)
            content, image_url = get_article_cafef(entry['id'])
            
            # Lấy thời gian đăng bài từ bài viết
            published_time = get_cafef_published_time(entry['id'])
            
			# Tóm tắt content
            model_name_list = list(model_dict.keys())
            for attempt in range(6):
                model_name = model_name_list[attempt % len(model_name_list)]
                try:
                    summary_content = summary_article(model_dict[model_name], content, news_type)
                    break
                except Exception as e:
                    if attempt == 5: raise Exception(f"Lỗi khi tóm tắt bài viết: {e}")
            
            full_news_df_dict[news_type].append({
                'source': 'CafeF',
                'title': entry['title'], 
                'content': summary_content,
                'image_url': image_url,
                'article_url': entry['id'],
                'published_time': published_time,
            })
            time.sleep(1)
        print(f"Thành công lấy tin từ CafeF: {category_name} ({url})")

Thành công lấy tin từ VietStock: vi_mo (https://vietstock.vn/761/kinh-te/vi-mo.rss)
Thành công lấy tin từ VietStock: tai_chinh_quoc_te (https://vietstock.vn/772/the-gioi/tai-chinh-quoc-te.rss)
Thành công lấy tin từ VietStock: chung_khoan_the_gioi (https://vietstock.vn/773/the-gioi/chung-khoan-the-gioi.rss)
Thành công lấy tin từ VietStock: hoat_dong_kinh_doanh (https://vietstock.vn/737/doanh-nghiep/hoat-dong-kinh-doanh.rss)
Thành công lấy tin từ CafeF: kinh_te_vi_mo (https://cafef.vn/vi-mo-dau-tu.chn)
Thành công lấy tin từ CafeF: ngan_hang (https://cafef.vn/tai-chinh-ngan-hang.chn)
Thành công lấy tin từ CafeF: chung_khoan (https://cafef.vn/thi-truong-chung-khoan.chn)
Thành công lấy tin từ CafeF: bat_dong_san (https://cafef.vn/bat-dong-san.chn)
Thành công lấy tin từ CafeF: tai_chinh_quoc_te (https://cafef.vn/tai-chinh-quoc-te.chn)
Thành công lấy tin từ CafeF: doanh_nghiep (https://cafef.vn/doanh-nghiep.chn)


In [ ]:
# Chuyển đổi dictionary thành pandas DataFrame
all_news_list = []
for news_type, articles in full_news_df_dict.items():
    for article in articles:
        # Thêm cột news_type vào mỗi article
        article_with_type = article.copy()
        article_with_type['news_type'] = news_type
        all_news_list.append(article_with_type)

# Tạo DataFrame
news_df = pd.DataFrame(all_news_list)
news_df['published_time'] = news_df['published_time'].apply(convert_published_time)

# Sắp xếp lại thứ tự các cột để news_type ở đầu
news_df = news_df[['news_type'] + [col for col in news_df.columns if col != 'news_type']]

# Đêm số từ trong bản tóm tắt
news_df['word_count'] = news_df['content'].str.split().str.len()

# Sử dụng hàm phân tích tác động tin tức
print("Đang phân tích tác động của tin tức đến thị trường chứng khoán Việt Nam...")
news_df['impact'] = analyze_news_impact(news_df, gemini_25_thinking)

Đang phân tích tác động của tin tức đến thị trường chứng khoán Việt Nam...
